# Práctica 2. Gestión y explotación de datos

### Realizada por Fernando Lucas Ruiz

## 1. Importar librerias y generar funciones

In [10]:
import mysql.connector
import pandas as pd

#importo estas librerias porque con pandas me sale siempre un warning incomodo para mostrar ante un usuario
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

Función para conectar con la base de datos

In [11]:
def conectar_mysql(host, user, password, database) :
    #Usamos el mecanismo de excepciones para controlar que todo es correcto
    try :
        mydb = mysql.connector.connect(
            host=host,
            user=user, #Usuario
            password=password,#contraseña
            database=database #esquema
        )
        return mydb
    except :
        return None

## 2. Consultas SQL

En estas lineas de código se especifican las consultas propuestas en la base de datos

In [12]:
# 1. Resultado agrupado de los pacientes por las propiedades sexo y rango de edad. 
# AYUDA: Se puede hacer más de un JOIN sobre la misma tabla. El resultado debería ser parecido a esta tabla:
consulta1 = """select p1.nombre as sexo, p2.nombre as 'rango de edad', count(m.codigo) as 'n muestras' from b3b2_muestra as m 
inner join (select muestra, nombre from b3b2_muestra_propiedad mp inner join b3b2_propiedad p on mp.propiedad = p.id where p.tipo = 'sexo') as p1 
on m.codigo=p1.muestra 
inner join 
(select muestra, nombre from b3b2_muestra_propiedad mp inner join b3b2_propiedad p on mp.propiedad = p.id where p.tipo = 'rango de edad') as p2
on p1.muestra = p2.muestra
group by p1.nombre, p2.nombre"""


# 2 Número de genes que participan en la rutas que contenta la palabra 'platelet'. 
# Simplemente hay que devolver el número.
consulta2 = """select count(g.gene_ensembl) as "numero de genes" from b3b2_gen as g
inner join b3b2_gen_pathway as gp on g.gene_ensembl=gp.gen
inner join b3b2_pathway as p on gp.pathway=p.codigo
where p.descripcion like '%platelet%'"""


# 3 Muestras de las que no tenemos resultados de expresión. Basta con devolver el código de la muestra.
consulta3 ="""select m.codigo as muestras from b3b2_muestra as m
left join b3b2_expresion as e on m.codigo = e.muestra
where e.expresion is null"""


# 4 Media de expresión de los genes secuenciados. Devolverá el gene_symbol y la media de expresión global.
consulta4 = """select g.gene_symbol, avg(e.expresion) as "Media expresion global" from b3b2_expresion as e
inner join b3b2_gen as g on e.gen = g.gene_ensembl
group by e.gen"""


# 5 Número de pathways de los genes que tienen media de expresión mayor de 25. Sólo debe devolver un número.
consulta5 = """select sum(t.cuenta) as "Número de pathways" from (
select count(p.codigo) as cuenta from b3b2_expresion as e
inner join b3b2_gen as g on e.gen = g.gene_ensembl
inner join b3b2_gen_pathway as gp on gp.gen=g.gene_ensembl
inner join b3b2_pathway as p on p.codigo=gp.pathway
group by e.gen
having avg(e.expresion) > 25
) as t"""


# 6 Muestras de las que no tenemos alguna propiedad (o sexo o rango de edad). Basta con devolver el código de la muestra.
consulta6 = """select m.codigo as codigo_muestra from b3b2_muestra as m
left join
    (select muestra from b3b2_muestra_propiedad as mp inner join b3b2_propiedad as p on mp.propiedad = p.id where p.tipo = 'sexo') as sexo
on m.codigo = sexo.muestra
left join  
    (select muestra from b3b2_muestra_propiedad as mp inner join b3b2_propiedad as p on mp.propiedad = p.id where p.tipo = 'rango de edad') as rango
on m.codigo = rango.muestra
where sexo.muestra is null or rango.muestra is null"""


# 7 Número de muestras por run.
consulta7 = """select r.nombre as run, count(m.codigo) as "n muestras" from b3b2_muestra as m
inner join b3b2_muestra_run as mr on m.codigo=mr.muestra
inner join b3b2_run as r on mr.run = r.nombre
group by r.nombre"""


# 8 Consulta que devuelva los genes cuyo log2foldchange sea mayor de 1 en valor absoluto entre muestras con sexo hombre y muestras con sexo mujer. 
# Deberá de devolver el gene_symbol y el valor de log2foldchange.
consulta8 = """select exp_mujer.gene_symbol, (log2(media_mujer/media_hombre)) as log2fc from
(select g.gene_symbol, avg(e.expresion) as media_mujer from b3b2_expresion as e
left join b3b2_gen as g on e.gen = g.gene_ensembl
inner join b3b2_muestra as m on e.muestra = m.codigo 
inner join b3b2_muestra_propiedad as mp on m.codigo = mp.muestra
inner join b3b2_propiedad as p on mp.propiedad = p.id where p.tipo = "sexo" and p.nombre = "Mujer"
group by e.gen) as exp_mujer
inner join 
(select g.gene_symbol, avg(e.expresion) as media_hombre from b3b2_expresion as e
left join b3b2_gen as g on e.gen = g.gene_ensembl
inner join b3b2_muestra as m on e.muestra = m.codigo 
inner join b3b2_muestra_propiedad as mp on m.codigo = mp.muestra
inner join b3b2_propiedad as p on mp.propiedad = p.id where p.tipo = "sexo" and p.nombre = "Hombre"
group by e.gen) as exp_hombre 
on exp_mujer.gene_symbol = exp_hombre.gene_symbol
where abs(log2(media_mujer/media_hombre)) > 1"""

## 3. Programa

Programa en el cual el usuario puede elegir entre las 8 consultas propuestas para la tarea.

In [13]:
while True:
     # hacemos conexión con la base de datos
     conexion = conectar_mysql("dayhoff.inf.um.es", "alubio11", "alubio11PW-", "alubio11")
     
     print("\n\n1. Consulta 1: Resultado agrupado de los pacientes por las propiedades sexo y rango de edad")
     print("2. Consulta 2: Número de genes que participan en la rutas que contenta la palabra 'platelet'")
     print("3. Consulta 3: Muestras de las que no tenemos resultados de expresión")
     print("4. Consulta 4: Media de expresión de los genes secuenciados")
     print("5. Consulta 5: Número de pathways de los genes que tienen media de expresión mayor de 25")
     print("6. Consulta 6: Muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)")
     print("7. Consulta 7: Número de muestras por run")
     print("8. Consulta 8: log2foldchange sea mayor de 1 en valor absoluto entre muestras con sexo hombre y muestras con sexo mujer\n")

     # input para que el usuario pueda elegir entre las 8 opciones
     opcion = input("\nIndique el opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8): ")
     opciones = ["1", "2", "3", "4", "5", "6", "7", "8"]
     
     # Según la opción que marque el usuario la variable tabla_consulta tendrá una consulta específica
     if opcion == "1":
          tabla_consulta = pd.read_sql(consulta1, conexion)
     
     elif opcion == "2":
          tabla_consulta = pd.read_sql(consulta2, conexion)

     elif opcion == "3":
          tabla_consulta = pd.read_sql(consulta3, conexion)

     elif opcion == "4":
          tabla_consulta = pd.read_sql(consulta4, conexion)

     elif opcion == "5":
          tabla_consulta = pd.read_sql(consulta5, conexion)
     
     elif opcion == "6":
          tabla_consulta = pd.read_sql(consulta6, conexion)

     elif opcion == "7":
          tabla_consulta = pd.read_sql(consulta7, conexion)
          
     elif opcion == "8":
          tabla_consulta = pd.read_sql(consulta8, conexion) 

     else: # si no elige entre las 8 opciones le volverá a preguntar al usuario
          print("Has elegido:", opcion, ". Por favor, elige entre las 8 opciones propuestas") 
          True

     # Si la opción dispuesta por el usuario no está dentro de la lista opciones antes creada, le vuelve a preguntar al usario
     if opcion in opciones:

          #Devuelve por pantalla la tabla de la consulta específica
          print("\n\n", tabla_consulta.to_string(index=False), "\n")

          # Preguntar al usuario si quiere seguir mirando opciones de consultas
          
          continuar = input("¿Quieres continuar? (y/n)")
          if continuar == "y":
               True
          elif continuar == "n":
               conexion.close() # Cerrar sesión 
               print("\n\n\t El programa ha finalizado correctamente.")
               break # para el programa
               
          break

     else:
          True
     
     
     




1. Consulta 1: Resultado agrupado de los pacientes por las propiedades sexo y rango de edad
2. Consulta 2: Número de genes que participan en la rutas que contenta la palabra 'platelet'
3. Consulta 3: Muestras de las que no tenemos resultados de expresión
4. Consulta 4: Media de expresión de los genes secuenciados
5. Consulta 5: Número de pathways de los genes que tienen media de expresión mayor de 25
6. Consulta 6: Muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
7. Consulta 7: Número de muestras por run
8. Consulta 8: log2foldchange sea mayor de 1 en valor absoluto entre muestras con sexo hombre y muestras con sexo mujer



   sexo rango de edad  n muestras
 Mujer           <18           7
Hombre         18-45           7
 Mujer         45-70           7
Hombre           >70           6 



	 El programa ha finalizado correctamente.
